# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [66]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,aa,aaa,aaaaah,aaaagh,aaaah,aaah,aaahhhhh,aah,aahh,aana,...,zillion,zillions,zombie,zombies,zone,zoning,zoo,zoomy,zurich,éclair
ali,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
amy,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,0,1,0,0,0,0,0,0,0,...,1,0,1,1,0,1,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eddie,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [68]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,amy,anthony,bill,bo,dave,eddie,george,hasan,jim,...,katt,louis,mike,patrice,reggie,richard,ricky,seth,tom,trevor
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
aaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
aaaaah,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaagh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
aaaah,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [70]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

/Users/karan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [71]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.005*"theres" + 0.004*"goes" + 0.004*"way" + 0.004*"cause" + 0.003*"house" + 0.003*"thing" + 0.003*"love" + 0.003*"okay" + 0.003*"mean" + 0.003*"tell"'),
 (1,
  '0.008*"fucking" + 0.006*"cause" + 0.004*"thing" + 0.004*"day" + 0.004*"didnt" + 0.004*"theres" + 0.004*"god" + 0.004*"went" + 0.004*"tell" + 0.003*"life"')]

In [72]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.007*"fucking" + 0.005*"theres" + 0.005*"god" + 0.004*"thing" + 0.004*"cause" + 0.004*"tell" + 0.004*"didnt" + 0.004*"day" + 0.003*"make" + 0.003*"way"'),
 (1,
  '0.007*"cause" + 0.005*"day" + 0.005*"went" + 0.004*"theres" + 0.004*"fucking" + 0.004*"okay" + 0.004*"love" + 0.004*"thing" + 0.003*"goes" + 0.003*"mean"'),
 (2,
  '0.006*"didnt" + 0.004*"ahah" + 0.004*"life" + 0.004*"fucking" + 0.004*"black" + 0.003*"dave" + 0.003*"lot" + 0.003*"guy" + 0.003*"thing" + 0.003*"cause"')]

In [73]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.008*"fucking" + 0.005*"theres" + 0.005*"goes" + 0.005*"thing" + 0.005*"went" + 0.005*"day" + 0.004*"god" + 0.004*"cause" + 0.004*"didnt" + 0.004*"way"'),
 (1,
  '0.005*"didnt" + 0.005*"theres" + 0.004*"thing" + 0.004*"life" + 0.004*"fcking" + 0.004*"cause" + 0.004*"fck" + 0.004*"love" + 0.004*"lot" + 0.004*"fucking"'),
 (2,
  '0.012*"cause" + 0.008*"aint" + 0.006*"bitch" + 0.006*"white" + 0.006*"ass" + 0.005*"love" + 0.004*"make" + 0.004*"mean" + 0.004*"mother" + 0.004*"woman"'),
 (3,
  '0.008*"motherfucker" + 0.006*"house" + 0.005*"cos" + 0.004*"aint" + 0.003*"dick" + 0.003*"gus" + 0.003*"ass" + 0.003*"icecream" + 0.003*"president" + 0.003*"big"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [74]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [75]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
ali,ali wong baby cobra full transcript scraps fr...
amy,amy schumer the leather special full transcri...
anthony,anthony jeselnik thoughts and prayers full tr...
bill,bill burr im sorry you feel that way full tra...
bo,page not found scraps from the loft \r\t\tski...
dave,dave chappelle the age of spin transcript scr...
eddie,eddie murphy delirious transcript scraps from...
george,george carlin its bad for ya full transcript ...
hasan,page not found scraps from the loft \r\t\tski...
jim,jim jefferies bare full transcript scraps fro...


In [76]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/karan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [77]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,ali baby cobra transcript scraps skip reviewsm...
amy,leather transcript scraps skip reviewsmovie tr...
anthony,anthony jeselnik thoughts prayers transcript s...
bill,bill sorry way transcript scraps skip reviewsm...
bo,page scraps skip reviewsmovie transcriptsstanl...
dave,chappelle age spin transcript scraps skip revi...
eddie,eddie murphy transcript scraps skip reviewsmov...
george,george transcript scraps skip reviewsmovie tra...
hasan,page scraps skip reviewsmovie transcriptsstanl...
jim,jim jefferies transcript scraps skip reviewsmo...


In [78]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [79]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like',
 'im',
 'just',
 'know',
 'dont',
 'thats',
 'people',
 'say',
 'youre',
 'gonna',
 'right',
 'got',
 'think',
 'come',
 'good',
 'oh',
 'time',
 'want',
 'man',
 'said',
 'hes',
 'going',
 'yeah',
 'really',
 'look',
 'did',
 'shit',
 'theyre',
 'fuck',
 'little']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
stop_words = list(stop_words)
# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aaaaah,aaaagh,aaahhhhh,aah,aana,aargh,abc,abcs,ability,aborigines,...,zero,zillion,zillions,zombie,zombies,zone,zoo,zoomy,zurich,éclair
ali,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
amy,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eddie,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [81]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.009*"thing" + 0.007*"day" + 0.007*"way" + 0.006*"guy" + 0.006*"gon" + 0.006*"life" + 0.005*"lot" + 0.005*"years" + 0.005*"things" + 0.005*"cause"'),
 (1,
  '0.011*"cause" + 0.008*"day" + 0.007*"gon" + 0.006*"ass" + 0.006*"women" + 0.006*"way" + 0.006*"life" + 0.005*"thing" + 0.005*"lot" + 0.005*"baby"')]

In [82]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.012*"thing" + 0.010*"day" + 0.009*"cause" + 0.008*"life" + 0.007*"way" + 0.007*"gon" + 0.007*"ass" + 0.006*"guy" + 0.005*"things" + 0.005*"joke"'),
 (1,
  '0.008*"cause" + 0.006*"way" + 0.006*"motherfucker" + 0.006*"day" + 0.006*"gon" + 0.006*"house" + 0.005*"woman" + 0.005*"bitch" + 0.005*"years" + 0.005*"women"'),
 (2,
  '0.008*"lot" + 0.007*"thing" + 0.006*"day" + 0.006*"guy" + 0.006*"god" + 0.006*"kids" + 0.006*"gon" + 0.006*"kind" + 0.005*"life" + 0.005*"way"')]

In [83]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.011*"ass" + 0.010*"gon" + 0.007*"cause" + 0.007*"mother" + 0.006*"house" + 0.006*"way" + 0.006*"motherfucker" + 0.006*"thing" + 0.005*"guy" + 0.005*"day"'),
 (1,
  '0.011*"cause" + 0.010*"life" + 0.009*"thing" + 0.007*"day" + 0.007*"women" + 0.007*"gon" + 0.006*"woman" + 0.006*"kids" + 0.006*"way" + 0.006*"things"'),
 (2,
  '0.012*"day" + 0.009*"cause" + 0.008*"thing" + 0.007*"women" + 0.006*"lot" + 0.006*"life" + 0.006*"baby" + 0.005*"way" + 0.005*"gon" + 0.005*"bitch"'),
 (3,
  '0.008*"thing" + 0.007*"way" + 0.006*"lot" + 0.006*"things" + 0.006*"day" + 0.006*"god" + 0.005*"years" + 0.005*"guy" + 0.005*"life" + 0.005*"comedy"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [84]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [85]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,ali wong baby cobra full transcript scraps lof...
amy,amy leather special full transcript scraps lof...
anthony,anthony jeselnik thoughts prayers full transcr...
bill,bill im sorry way full transcript scraps loft ...
bo,page scraps loft skip moviesmovie reviewsmovie...
dave,chappelle age spin transcript scraps loft skip...
eddie,eddie murphy delirious transcript scraps loft ...
george,george bad ya full transcript scraps loft skip...
hasan,page scraps loft skip moviesmovie reviewsmovie...
jim,jim jefferies full transcript scraps loft skip...


In [86]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aa,aaa,aaaaah,aaaagh,aaaah,aaahhhhh,aah,aana,aargh,abc,...,zero,zillion,zillions,zombie,zombies,zone,zoo,zoomy,zurich,éclair
ali,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
amy,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eddie,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [88]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"cause" + 0.005*"fucking" + 0.005*"kids" + 0.004*"kid" + 0.004*"hey" + 0.004*"joke" + 0.003*"id" + 0.003*"children" + 0.003*"motherfucker" + 0.003*"guys"'),
 (1,
  '0.009*"cause" + 0.005*"ass" + 0.004*"kids" + 0.004*"okay" + 0.004*"bitch" + 0.004*"car" + 0.003*"ta" + 0.003*"hey" + 0.003*"men" + 0.003*"guys"')]

In [89]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.013*"cause" + 0.007*"ass" + 0.006*"kids" + 0.005*"bitch" + 0.004*"ta" + 0.004*"fck" + 0.004*"dog" + 0.004*"mother" + 0.004*"aint" + 0.004*"girl"'),
 (1,
  '0.007*"fucking" + 0.005*"kid" + 0.005*"joke" + 0.005*"kids" + 0.004*"children" + 0.004*"ass" + 0.004*"cause" + 0.004*"id" + 0.004*"motherfucker" + 0.004*"dude"'),
 (2,
  '0.005*"cause" + 0.005*"hey" + 0.004*"okay" + 0.004*"guys" + 0.004*"friend" + 0.004*"id" + 0.003*"fucking" + 0.003*"point" + 0.003*"kids" + 0.003*"dad"')]

In [90]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.007*"motherfucker" + 0.007*"kids" + 0.005*"cause" + 0.005*"bitch" + 0.004*"fucking" + 0.004*"children" + 0.003*"kid" + 0.003*"car" + 0.003*"guys" + 0.003*"ass"'),
 (1,
  '0.008*"cause" + 0.006*"id" + 0.004*"uh" + 0.004*"girl" + 0.004*"men" + 0.004*"point" + 0.003*"guys" + 0.003*"fucking" + 0.003*"joke" + 0.003*"okay"'),
 (2,
  '0.012*"ass" + 0.011*"cause" + 0.008*"mother" + 0.006*"dude" + 0.005*"fucking" + 0.005*"fucker" + 0.004*"hey" + 0.004*"everybody" + 0.004*"dog" + 0.004*"kids"'),
 (3,
  '0.006*"cause" + 0.006*"kids" + 0.005*"fck" + 0.005*"hey" + 0.005*"dad" + 0.004*"kid" + 0.004*"dude" + 0.004*"mom" + 0.003*"ta" + 0.003*"na"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [91]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.013*"fck" + 0.007*"fcking" + 0.006*"kids" + 0.005*"cause" + 0.005*"dude" + 0.004*"girl" + 0.004*"everybody" + 0.004*"different" + 0.004*"stupid" + 0.004*"kid"'),
 (1,
  '0.008*"fucking" + 0.005*"children" + 0.004*"guns" + 0.004*"kid" + 0.004*"men" + 0.004*"ass" + 0.004*"child" + 0.004*"cause" + 0.004*"gun" + 0.004*"girl"'),
 (2,
  '0.008*"cause" + 0.005*"hey" + 0.005*"id" + 0.005*"kids" + 0.004*"guys" + 0.004*"joke" + 0.004*"okay" + 0.004*"kid" + 0.004*"fucking" + 0.003*"dad"'),
 (3,
  '0.010*"ass" + 0.009*"motherfucker" + 0.009*"cause" + 0.007*"bitch" + 0.006*"mother" + 0.005*"kids" + 0.005*"aint" + 0.004*"ta" + 0.004*"car" + 0.004*"dog"')]

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: mom, parents
* Topic 1: husband, wife
* Topic 2: guns
* Topic 3: profanity

In [92]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
topic_probabilities = []
for doc in corpus_transformed:
    topic_probabilities.append(max(doc, key=lambda item: item[1]))
list(zip(topic_probabilities, data_dtmna.index))


[((3, 0.99943995), 'ali'),
 ((1, 0.99920493), 'amy'),
 ((2, 0.9993687), 'anthony'),
 ((2, 0.9996065), 'bill'),
 ((3, 0.8917563), 'bo'),
 ((3, 0.9995177), 'dave'),
 ((3, 0.99952793), 'eddie'),
 ((1, 0.9996013), 'george'),
 ((3, 0.89175916), 'hasan'),
 ((1, 0.99957734), 'jim'),
 ((0, 0.9995848), 'joe'),
 ((2, 0.99956673), 'john'),
 ((3, 0.99959105), 'katt'),
 ((2, 0.9994016), 'louis'),
 ((2, 0.99959886), 'mike'),
 ((2, 0.9995458), 'patrice'),
 ((2, 0.9991157), 'reggie'),
 ((3, 0.9995243), 'richard'),
 ((2, 0.99962), 'ricky'),
 ((1, 0.9989631), 'seth'),
 ((2, 0.9995113), 'tom'),
 ((2, 0.9995042), 'trevor')]

For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: mom, parents [Anthony, Hasan, Louis, Ricky]
* Topic 1: husband, wife [Ali, John, Mike]
* Topic 2: guns [Bill, Bo, Jim]
* Topic 3: profanity [Dave, Joe]

### Assignment:
1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

In [102]:
ldana1 = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=100, alpha=0.5, eta=0.01, minimum_probability=0.0, chunksize=100)
ldana1.print_topics()


[(0,
  '0.007*"ahah" + 0.007*"joke" + 0.006*"love" + 0.006*"anthony" + 0.005*"ok" + 0.005*"ta" + 0.005*"fucking" + 0.005*"school" + 0.005*"hey" + 0.005*"mad"'),
 (1,
  '0.011*"cause" + 0.010*"ass" + 0.007*"kids" + 0.006*"mother" + 0.006*"fucking" + 0.005*"motherfucker" + 0.005*"id" + 0.005*"dog" + 0.005*"hey" + 0.004*"kid"'),
 (2,
  '0.009*"cause" + 0.006*"bitch" + 0.005*"kids" + 0.005*"fck" + 0.005*"ta" + 0.005*"girl" + 0.004*"president" + 0.004*"okay" + 0.004*"different" + 0.004*"job"'),
 (3,
  '0.008*"hey" + 0.007*"kid" + 0.007*"fucking" + 0.007*"okay" + 0.007*"cause" + 0.006*"dude" + 0.006*"gun" + 0.006*"kids" + 0.006*"guys" + 0.006*"clinton"')]

In [ ]:
These four topics look pretty decent. Let's settle on these for now.
* Topic 0: love
* Topic 1: kids
* Topic 2: girl,job
* Topic 3: gun

In [103]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana1[corpusna]
topic_probabilities = []
for doc in corpus_transformed:
    topic_probabilities.append(max(doc, key=lambda item: item[1]))
list(zip(topic_probabilities, data_dtmna.index))

[((0, 0.80900043), 'ali'),
 ((3, 0.99786156), 'amy'),
 ((0, 0.99831134), 'anthony'),
 ((3, 0.99898154), 'bill'),
 ((0, 0.8083804), 'bo'),
 ((0, 0.9986985), 'dave'),
 ((1, 0.99876106), 'eddie'),
 ((1, 0.9989789), 'george'),
 ((0, 0.80837876), 'hasan'),
 ((1, 0.99882597), 'jim'),
 ((2, 0.9226441), 'joe'),
 ((3, 0.9989281), 'john'),
 ((2, 0.99885535), 'katt'),
 ((1, 0.9982853), 'louis'),
 ((1, 0.9989497), 'mike'),
 ((2, 0.998774), 'patrice'),
 ((0, 0.9978687), 'reggie'),
 ((1, 0.9988058), 'richard'),
 ((2, 0.99896723), 'ricky'),
 ((2, 0.99736124), 'seth'),
 ((1, 0.9985479), 'tom'),
 ((2, 0.99873054), 'trevor')]

In [ ]:
For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: love [Ali, Anthony, Bo, Dave, Hasan, Reggie]
* Topic 1: kids [Eddie,George,Jim,Louis,Mike,Richard,Tom]
* Topic 2: girl,job [Joe,Katt,Patrice, Ricky, Seth, Trevor]
* Topic 3: gun [Amy, John,Bill,]

In [140]:
# Let's create a function to pull out verbs and adverbs from a string of text

def extract_nouns_and_verbs(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and verbs.'''
    # Define the set of POS tags to include
    pos_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    # Define the set of words to exclude
    stop_words = set(stopwords.words('english')) - {'not', 'no'}
    # Tokenize the text and tag the words with their POS
    tokenized = word_tokenize(text)
    tagged = pos_tag(tokenized)
    # Extract the nouns and verbs and exclude the stop words and certain POS tags
    nouns_verbs = [word for (word, pos) in tagged if pos in pos_tags and word not in stop_words]
    return ' '.join(nouns_verbs)


In [141]:
data_v_adv = pd.DataFrame(data_clean.transcript.apply(v_adv))
data_v_adv

,transcript
ali,content carlindave chappelleinterviewsplayboy ...
amy,schumer content carlindave chappelleinterviews...
anthony,and content carlindave chappelleinterviewsplay...
bill,burr feel content carlindave chappelleintervie...
bo,found content carlindave chappelleinterviewspl...
dave,dave content carlindave chappelleinterviewspla...
eddie,content carlindave chappelleinterviewsplayboy ...
george,carlin content carlindave chappelleinterviewsp...
hasan,found content carlindave chappelleinterviewspl...
jim,bare content carlindave chappelleinterviewspla...


In [142]:
# Create a new document-term matrix using only verbs and adverbs, also remove common words with max_df
cvva = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvva = cvva.fit_transform(data_v_adv.transcript)
data_dtmva = pd.DataFrame(data_cvva.toarray(), columns=cvva.get_feature_names_out())
data_dtmva.index = data_v_adv.index
data_dtmva


,aaah,aahh,abort,absorb,absorbing,abuse,abused,accept,accepted,accepting,...,youd,youi,youll,youve,youyou,youyoull,yummy,zach,zero,zoning
ali,0,0,0,0,1,1,1,0,0,0,...,0,1,2,0,0,0,0,0,0,0
amy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
bill,0,0,0,1,0,0,0,0,0,0,...,0,0,2,1,0,0,0,0,0,1
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,1,0,0,0,0,0,0,0,0,0,...,2,0,1,2,0,0,0,0,0,0
eddie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,0,0,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [143]:
# Create the gensim corpus
corpusva = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmva.transpose()))

# Create the vocabulary dictionary
id2wordva = dict((v, k) for k, v in cvva.vocabulary_.items())

In [144]:
ldanva = models.LdaModel(corpus=corpusva, num_topics=4, id2word=id2wordva, passes=100, alpha=0.5, eta=0.01, minimum_probability=0.0, chunksize=100)
ldanva.print_topics()

[(0,
  '0.015*"aint" + 0.012*"guys" + 0.009*"wan" + 0.008*"motherfucking" + 0.008*"cause" + 0.008*"hate" + 0.008*"understand" + 0.007*"eat" + 0.007*"die" + 0.006*"bitch"'),
 (1,
  '0.015*"walked" + 0.011*"looked" + 0.010*"knew" + 0.009*"met" + 0.009*"walking" + 0.009*"fucking" + 0.008*"speak" + 0.008*"understand" + 0.007*"meet" + 0.007*"hed"'),
 (2,
  '0.018*"fucking" + 0.010*"cause" + 0.009*"fcking" + 0.008*"thinking" + 0.007*"run" + 0.007*"die" + 0.007*"guys" + 0.007*"kill" + 0.006*"whats" + 0.006*"sitting"'),
 (3,
  '0.035*"fucking" + 0.008*"understand" + 0.008*"bring" + 0.007*"hate" + 0.006*"fucked" + 0.006*"hed" + 0.006*"kill" + 0.006*"god" + 0.006*"cause" + 0.006*"shut"')]

In [100]:
These four topics look pretty decent. Let's settle on these for now.
* Topic 0: eat
* Topic 1: walk
* Topic 2: kill
* Topic 3: god

SyntaxError: invalid syntax (2776497114.py, line 1)

In [145]:
# Let's take a look at which topics each transcript contains
corpus_transformed1 = ldanva[corpusva]
topic_probabilities = []
for doc in corpus_transformed:
    topic_probabilities.append(max(doc, key=lambda item: item[1]))
list(zip(topic_probabilities, data_dtmva.index))

[((0, 0.8090003), 'ali'),
 ((3, 0.99786156), 'amy'),
 ((0, 0.99831206), 'anthony'),
 ((3, 0.9989816), 'bill'),
 ((0, 0.80837756), 'bo'),
 ((0, 0.9986979), 'dave'),
 ((1, 0.9987608), 'eddie'),
 ((1, 0.9989789), 'george'),
 ((0, 0.80837566), 'hasan'),
 ((1, 0.9988258), 'jim'),
 ((2, 0.9226444), 'joe'),
 ((3, 0.9989277), 'john'),
 ((2, 0.9988554), 'katt'),
 ((1, 0.99828523), 'louis'),
 ((1, 0.9989493), 'mike'),
 ((2, 0.99877423), 'patrice'),
 ((0, 0.99786866), 'reggie'),
 ((1, 0.99880576), 'richard'),
 ((2, 0.9989672), 'ricky'),
 ((2, 0.9973614), 'seth'),
 ((1, 0.99854696), 'tom'),
 ((2, 0.9987306), 'trevor')]

In [ ]:
For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: love [Ali, Anthony, Bo, Dave, Hasan, Reggie]
* Topic 1: kids [Eddie,George,Jim,Louis,Mike,Richard,Tom]
* Topic 2: girl,job [Joe,Katt,Patrice, Ricky, Seth, Trevor]
* Topic 3: gun [Amy, John,Bill,]